<a href="https://colab.research.google.com/github/Munya07/NLP/blob/master/2.NER/Elmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Keras==1.0.6
import numpy as np
from keras.layers.recurrent import LSTM
from keras.layers.core import  Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import tensorflow_hub as hub
from keras.layers import Dense,TimeDistributed
import keras.layers as layers
from keras.models import Model,Sequential
import tensorflow as tf
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import *

Using TensorFlow backend.
W0613 03:52:48.420124 140684539537280 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [6]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd /content/drive/My\ Drive/NLP/Task2/
!ls
raw = open('wikigold.conll.txt', 'r').readlines()
 
all_x = []
point = []
for line in raw:
    stripped_line = line.strip().split(' ')
    point.append(stripped_line)
    if line == '\n':
        all_x.append(point[:-1])
        point = []
all_x = all_x[:-1]
 
lengths = [len(x) for x in all_x]
print('Input sequence length range: ', max(lengths), min(lengths))
 
short_x = [x for x in all_x if len(x) < 64]
 
X = [[c[0] for c in x] for x in short_x]
y = [[c[1] for c in y] for y in short_x]
 
all_text = [c for x in X for c in x]


Mounted at /content/drive
/content/drive/My Drive/NLP/Task2
wikigold.conll.txt
Input sequence length range:  144 1


In [7]:
words = list(set(all_text))
word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}
labels = list(set([c for x in y for c in x]))
label2ind = {label: (index + 1) for index, label in enumerate(labels)}
ind2label = {(index + 1): label for index, label in enumerate(labels)}
print('Vocabulary size:', len(word2ind), len(label2ind))

Vocabulary size: 8285 5


In [8]:
maxlen = max([len(x) for x in X])
print('Maximum sequence length:', maxlen)
print(label2ind)

Maximum sequence length: 63
{'I-ORG': 1, 'I-MISC': 2, 'O': 3, 'I-PER': 4, 'I-LOC': 5}


In [0]:
def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result
 
X_enc = [[word2ind[c] for c in x] for x in X]
max_label = max(label2ind.values()) + 1
y_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in y]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]
 
X_enc = pad_sequences(X_enc, maxlen=maxlen)
y_enc = pad_sequences(y_enc, maxlen=maxlen)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=11*32, train_size=45*32, random_state=42)
print('Training and testing tensor shapes:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)
 
max_features = len(word2ind)
embedding_size = 300
hidden_size = 32
out_size = len(label2ind) + 1
batch_size = 32

Training and testing tensor shapes: (1440, 63) (352, 63) (1440, 63, 6) (352, 63, 6)


In [11]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

Instructions for updating:
Colocations handled automatically by placer.


W0613 04:02:32.744062 140684539537280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [0]:

class ELMoEmbedding(Layer):

    def __init__(self, idx2word, output_mode="default", trainable=True, **kwargs):
        assert output_mode in ["default", "word_emb", "lstm_outputs1", "lstm_outputs2", "elmo"]
        assert trainable in [True, False]
        self.idx2word = idx2word
        self.output_mode = output_mode
        self.trainable = trainable
        self.max_length = None
        self.word_mapping = None
        self.lookup_table = None
        self.elmo_model = None
        self.embedding = None
        super(ELMoEmbedding, self).__init__(**kwargs)

    def build(self, input_shape):
        self.max_length = input_shape[1]
        self.word_mapping = [x[1] for x in sorted(self.idx2word.items(), key=lambda x: x[0])]
        self.lookup_table = tf.contrib.lookup.index_to_string_table_from_tensor(self.word_mapping, default_value="<UNK>")
        self.lookup_table.init.run(session=K.get_session())
        self.elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=self.trainable)
        super(ELMoEmbedding, self).build(input_shape)

    def call(self, x):
        x = tf.cast(x, dtype=tf.int64)
        sequence_lengths = tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32)
        strings = tf.squeeze(self.lookup_table.lookup(x))
        inputs = {
            "tokens": strings,
            "sequence_len": sequence_lengths
        }
        return self.elmo_model(inputs, signature="tokens", as_dict=True)[self.output_mode]

    def compute_output_shape(self, input_shape):
        if self.output_mode == "default":
            return (input_shape[0], 1024)
        if self.output_mode == "word_emb":
            return (input_shape[0], self.max_length, 512)
        if self.output_mode == "lstm_outputs1":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "lstm_outputs2":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "elmo":
            return (input_shape[0], self.max_length, 1024)

    def get_config(self):
        config = {
            'idx2word': self.idx2word,
            'output_mode': self.output_mode 
        }
        return list(config.items())


In [14]:
sentence_input = Input(shape=(X_train.shape[1],), dtype=tf.int64)
sentence_embedding = ELMoEmbedding(idx2word=ind2word, output_mode="elmo", trainable=True)(sentence_input) # These two are interchangeable


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.


W0613 04:03:10.720901 140684539537280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/lookup_ops.py:1318: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use `initializer` instead.


W0613 04:03:10.737480 140684539537280 deprecation.py:323] From <ipython-input-13-316b88bffc30>:21: InitializableLookupTableBase.init (from tensorflow.python.ops.lookup_ops) is deprecated and will be removed after 2018-12-15.
Instructions for updating:
Use `initializer` instead.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0613 04:03:12.528014 140684539537280 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [15]:
dropout = Dropout(0.5)(sentence_embedding)
lstm_ = LSTM(hidden_size,batch_size=batch_size, return_sequences=True)(sentence_embedding)
timed_ = layers.TimeDistributed(layers.Dense(out_size))(lstm_)
pred = layers.Activation('softmax')(timed_)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0613 04:03:18.036671 140684539537280 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model = Model(inputs=[sentence_input], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 63)                0         
_________________________________________________________________
el_mo_embedding_1 (ELMoEmbed (None, 63, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 63, 32)            135296    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 63, 6)             198       
_________________________________________________________________
activation_1 (Activation)    (None, 63, 6)             0         
Total params: 135,494
Trainable params: 135,494
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Raw test score:', score)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0613 04:03:27.985853 140684539537280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


W0613 04:03:28.145997 140684539537280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:102: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Train on 1440 samples, validate on 352 samples
Epoch 1/10
1440/1440 [==============================] - 719s 499ms/step - loss: 0.5761 - val_loss: 0.2793
Epoch 2/10
1440/1440 [==============================] - 723s 502ms/step - loss: 0.2171 - val_loss: 0.1758
Epoch 3/10
1440/1440 [==============================] - 710s 493ms/step - loss: 0.1552 - val_loss: 0.1393
Epoch 4/10
1440/1440 [==============================] - 706s 490ms/step - loss: 0.1276 - val_loss: 0.1186
Epoch 5/10
1440/1440 [==============================] - 723s 502ms/step - loss: 0.1094 - val_loss: 0.1029
Epoch 6/10
1440/1440 [==============================] - 713s 495ms/step - loss: 0.0952 - val_loss: 0.0917
Epoch 7/10
1440/1440 [==============================] - 709s 493ms/step - loss: 0.0835 - val_loss: 0.0829
Epoch 8/10
1440/1440 [==============================] - 702s 487ms/step - loss: 0.0743 - val_loss: 0.0766
Epoch 9/10
1440/1440 [==============================] - 708s 492ms/step - loss: 0.0668 - val_loss: 0.0721

In [0]:
def score(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

In [0]:
y_prob = model.predict(X_train) 
y_classes = y_prob.argmax(axis=-1)

In [22]:
y_classes

array([[0, 0, 0, ..., 3, 3, 3],
       [0, 0, 0, ..., 3, 3, 3],
       [0, 0, 0, ..., 0, 0, 3],
       ...,
       [0, 0, 0, ..., 3, 3, 3],
       [0, 0, 0, ..., 4, 5, 3],
       [0, 0, 0, ..., 3, 4, 3]])

In [23]:
y_classes[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3])

In [0]:
yh = y_train.argmax(2)

In [26]:
pr=y_classes
fyh, fpr = score(yh, pr)
print('Training accuracy:', accuracy_score(fyh, fpr))
print('Training confusion matrix:')
print(confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

Training accuracy: 0.9508362557323982
Training confusion matrix:
[[    0     0     0     0     0     0]
 [    0  1121    52   238    55    76]
 [    0   121   587   246    42    24]
 [    2   110    49 24547    20    24]
 [    0    26     9    52  1121    18]
 [    0   109    24    80    81   822]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(array([0.        , 0.75386685, 0.81414702, 0.97551961, 0.84988628,
        0.8526971 ]),
 array([0.        , 0.72697795, 0.5754902 , 0.99171784, 0.91435563,
        0.73655914]),
 array([0.        , 0.74017828, 0.6743251 , 0.98355204, 0.88094303,
        0.79038462]),
 array([    0,  1542,  1020, 24752,  1226,  1116]))

In [27]:
y_prob = model.predict(X_test) 
pr = y_prob.argmax(axis=-1)
yh = y_test.argmax(2)
fyh, fpr = score(yh, pr)
print('Testing accuracy:', accuracy_score(fyh, fpr))
print('Testing confusion matrix:')
print(confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

Testing accuracy: 0.9348679756262694
Testing confusion matrix:
[[   0    0    0    0    0    0]
 [   0  223   18   70   31   15]
 [   0   59  123   72    8   11]
 [   1   33   14 6138   11    7]
 [   0   16   10    9  237   11]
 [   0   32   10   17   26  183]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(array([0.        , 0.61432507, 0.70285714, 0.97335871, 0.7571885 ,
        0.8061674 ]),
 array([0.        , 0.62464986, 0.45054945, 0.9893617 , 0.83745583,
        0.68283582]),
 array([0.        , 0.61944444, 0.54910714, 0.98129496, 0.79530201,
        0.73939394]),
 array([   0,  357,  273, 6204,  283,  268]))